# サーバーへのアクセス

PCを同じネットワークに接続し、コマンドラインから以下でアクセスできる。
```
ssh peritec@iotschool
```

リモートからもアクセスできるようにしているがここでは省略\
社内サーバーの以下参照\
(192.168.100.5\disk1\03_個人フォルダ\mizunashi\AIサーバーリモートアクセス.txt)

---
# 設定ファイル更新

```
cd nginx-nr
cp -r BaseUserFolder [新規ユーザー名]
nano docker-compose.yml   #修正内容は下参照
cd nginx
nano default.conf   #修正内容は下参照
cd ../[新規ユーザー名]
nano settings.js    #修正内容は下参照
```
※ユーザー名は小文字にする


## docker-compose.yml ファイルの更新

docker-compose.ymlファイルに他のユーザー設定と同じように新規ユーザー情報を追加する。
```
[新規ユーザー名]:
  volumes:
    - type: volume
      source: data_[新規ユーザー名]
      target: /data
  build:
      context : ./[新規ユーザー名]/
      dockerfile: Dockerfile
  container_name: [新規ユーザー名]
  ports:
    - [新規ポート]:[新規ポート]
  extra_hosts:
    - "peritec.net:192.168.254.245"
```
一番下のvolumesにも追加
```
data_[新規ユーザー名]
```

## default.conf ファイルの更新

default.confファイルに他のユーザー設定と同じように新規ユーザー情報を追加する。
```
  location /[新規ユーザー名]/ {
    proxy_http_version 1.1;
    proxy_set_header Host              $host;
    proxy_set_header Upgrade $http_upgrade;
    proxy_set_header Connection "upgrade";

    proxy_pass        http://[新規ユーザー名]:[新規ポート]/;
  }
```

## settings.confファイルの更新

settings.jsファイルを更新する
```
username: "[新規ユーザー名]"
password: "[パスワード文字列]"
```
```
uiPort: process.env.PORT || [新規ポート],
```
パスワードの生成は、以下コマンド後にパスワードを入力するとパスワード文字列を生成できる。
```
node-red admin hash-pw
```


##変更反映

設定更新後、nginx-nr フォルダで以下コマンドを実行して  
Dockerを再起動する。
```
docker compose down --rmi all --volumes --remove-orphans
docker compose up -d --build
```